**Note**: You might experience slightly different interactions between the agents. The engineer agent may write incorrect code, which the executor agent will report and send back for correction. This process could go through multiple rounds.

In [2]:
#updated Code with Agents Handling the Entire Workflow:

llm_config = {"model": "gpt-4-turbo"}

from autogen import ConversableAgent

# Simulated policy database
policy_database = {
    "12345": {"status": "active", "customer_name": "John Doe"},
    "67890": {"status": "expired", "customer_name": "Jane Smith"},
    "54321": {"status": "active", "customer_name": "Alice Brown"},
}

# Function to simulate the validation in the agent
def validate_policy(policy_number):
    policy_info = policy_database.get(policy_number)
    if policy_info:
        if policy_info["status"] == "active":
            return "VALID"
        else:
            return "INVALID"
    else:
        return "INVALID"

# Agent to gather customer claim details
insurance_claim_info_agent = ConversableAgent(
    name="Insurance Claim Info Agent",
    system_message='''You are a helpful insurance agent here to assist
    customers with filing their insurance claims. 
    Your job is to gather the customer's policy number and details of the claim.
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

# Validation agent to validate policy status
insurance_claim_validation_agent = ConversableAgent(
    name="Insurance Claim Validation Agent",
    system_message='''You are an insurance claim validation agent. 
    You will check the policy number provided by the customer and 
    validate whether the policy is active or expired. Return 'VALID' for 
    active policies and 'INVALID - Policy Expired' for expired policies.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

# Agent to process the claim after validation
insurance_claim_processing_agent = ConversableAgent(
    name="Insurance Claim Processing Agent",
    system_message='''You are an insurance claims processing agent.
    After the claim has been validated, you will process the claim by updating
    the system with the claim details.
    Return 'PROCESSED' once done.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

# Proxy agent to simulate human interaction
customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
)

# Simulate the claim and validation process using agents
def initiate_claim_workflow():
    chats = [
        # Step 1: Gather claim details
        {
            "sender": insurance_claim_info_agent,
            "recipient": customer_proxy_agent,
            "message": "Please provide your policy number and claim details.",
            "summary_method": "reflection_with_llm",
            "summary_args": {
                "summary_prompt": "Return the policy number and claim details as a JSON object: {'policy_number': '', 'claim_details': ''}",
            },
            "max_turns": 2,
        },
        # Step 2: Validate the policy
        {
            "sender": insurance_claim_validation_agent,
            "recipient": customer_proxy_agent,
            "message": "Let's validate the policy number provided.",
            "summary_method": "reflection_with_llm",
            "summary_args": {
                "summary_prompt": "Return the policy validation result: 'VALID' or 'INVALID {'Please check your policy no or connect with your Insurance Advisor'}'.",
            },
            "max_turns": 1,
        },
        # Step 3: Process the claim if validation is valid
        {
            "sender": insurance_claim_processing_agent,
            "recipient": customer_proxy_agent,
            "message": "The claim has been validated. Processing the claim now if VALID.",
            "max_turns": 1,
            "summary_method": "reflection_with_llm",
        },
    ]

    from autogen import initiate_chats
    chat_results = initiate_chats(chats)

    # Output processing result and costs
    for chat_result in chat_results:
        print(chat_result.summary)
        print("\n")
        


# Example workflow initiation
initiate_claim_workflow()



********************************************************************************
Starting a new chat....

********************************************************************************
Insurance Claim Info Agent (to customer_proxy_agent):

Please provide your policy number and claim details.

--------------------------------------------------------------------------------


/usr/local/lib/python3.11/site-packages/autogen/agentchat/chat.py:47: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(


Provide feedback to Insurance Claim Info Agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: 12345
customer_proxy_agent (to Insurance Claim Info Agent):

12345

--------------------------------------------------------------------------------
Insurance Claim Info Agent (to customer_proxy_agent):

Thank you for providing your policy number (12345). Could you please share the details of your claim now?

--------------------------------------------------------------------------------
Provide feedback to Insurance Claim Info Agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: Building Insurance
customer_proxy_agent (to Insurance Claim Info Agent):

Building Insurance

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

**********************************************************************

In [5]:
#Updated Code with Agents Handling the Entire Workflow:

llm_config = {"model": "gpt-4-turbo"}

from autogen import ConversableAgent

# Simulated policy database
policy_database = {
    "12345": {"status": "active", "customer_name": "John Doe"},
    "67890": {"status": "expired", "customer_name": "Jane Smith"},
    "54321": {"status": "active", "customer_name": "Alice Brown"},
}

# Function to simulate the validation in the agent
def validate_policy(policy_number):
    policy_info = policy_database.get(policy_number)
    if policy_info:
        if policy_info["status"] == "active":
            return "VALID"
        else:
            return "INVALID - Policy Expired"
    else:
        return "INVALID - Policy Not Found"

# Agent to gather customer claim details
insurance_claim_info_agent = ConversableAgent(
    name="Insurance Claim Info Agent",
    system_message='''You are a helpful insurance agent here to assist
    customers with filing their insurance claims. 
    Your job is to gather the customer's policy number and details of the claim.
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

# Validation agent to validate policy status
insurance_claim_validation_agent = ConversableAgent(
    name="Insurance Claim Validation Agent",
    system_message='''You are an insurance claim validation agent. 
    You will check the policy number provided by the customer and 
    validate whether the policy is active or expired. Return 'VALID' for 
    active policies and 'INVALID - Policy Expired' for expired policies.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

# Agent to process the claim after validation
insurance_claim_processing_agent = ConversableAgent(
    name="Insurance Claim Processing Agent",
    system_message='''You are an insurance claims processing agent.
    After the claim has been validated, you will process the claim by updating
    the system with the claim details.
    Return 'PROCESSED' once done.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

# Proxy agent to simulate human interaction
customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
)

# Simulate the claim and validation process using agents
def initiate_claim_workflow():
    chats = [
        # Step 1: Gather claim details
        {
            "sender": insurance_claim_info_agent,
            "recipient": customer_proxy_agent,
            "message": "Please provide your policy number and claim details.",
            "summary_method": "reflection_with_llm",
            "summary_args": {
                "summary_prompt": "Return the policy number and claim details as a JSON object: {'policy_number': '', 'claim_details': ''}",
            },
            "max_turns": 2,
        },
        # Step 2: Validate the policy
        {
            "sender": insurance_claim_validation_agent,
            "recipient": customer_proxy_agent,
            "message": "Let's validate the policy number provided.",
            "summary_method": "reflection_with_llm",
            "summary_args": {
                "summary_prompt": "Return the policy validation result: 'VALID' or 'INVALID'.",
            },
            "max_turns": 1,
        },
        # Step 3: Process the claim if validation is valid
        {
            "sender": insurance_claim_processing_agent,
            "recipient": customer_proxy_agent,
            "message": "The claim has been validated. Processing the claim now.",
            "max_turns": 1,
            "summary_method": "reflection_with_llm",
        },
    ]

    from autogen import initiate_chats
    chat_results = initiate_chats(chats)

    # Output processing result and costs
    for chat_result in chat_results:
        print(chat_result.summary)
        print("\n")


# Example workflow initiation
initiate_claim_workflow()




********************************************************************************
Starting a new chat....

********************************************************************************
Insurance Claim Info Agent (to customer_proxy_agent):

Please provide your policy number and claim details.

--------------------------------------------------------------------------------
Provide feedback to Insurance Claim Info Agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: 54321
customer_proxy_agent (to Insurance Claim Info Agent):

54321

--------------------------------------------------------------------------------
Insurance Claim Info Agent (to customer_proxy_agent):

Thank you for providing your policy number. Can you now provide the details of the claim?

--------------------------------------------------------------------------------
Provide feedback to Insurance Claim Info Agent. Press enter to skip and use auto-reply, or type 'exit' to end the conv